In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
class BookSelection:
    if __name__ == "__main__":
        def __init__(self, url, search_keywords):
            self.url = url
            self.search_keywords = search_keywords

        def set_drivers(self):
            try:
                service = Service(executable_path=ChromeDriverManager().install())
                options = Options()
                # 참고 = https://codingstep.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%ED%81%AC%EB%A1%AC-%EC%98%B5%EC%85%98-python-selenium-chrome-optionsFeat-headless-window-size-user-agent-etc
                #지정한 user-agent로 설정합니다.
#                 user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
#                 options.add_argument('user-agent=' + user_agent)
                options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
#                 options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
#                 options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
#                 options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
#                 options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
#                 options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
#                 options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
                driver = webdriver.Chrome(service=service, options=options)  ##버전에 상관없이 현재 OS에 설치된 크롬브라우저 사용
                return driver
            except Exception as e:
                print(e)
        
        def get_url(self, driver):
            try:
                driver.get(self.url)
                driver.implicitly_wait(time_to_wait=10)
            except Exception as e:
                print(e)
                driver.quit()
                
        def get_searchKeywords(self, driver):
            try:
                search_input = driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/div[1]/form[2]/div/div/input")
                search_input.send_keys(self.search_keywords)
                driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/div[1]/form[2]/div/input").click()
                driver.implicitly_wait(time_to_wait=10)
            except Exception as e:
                print(e)
                driver.quit()
        
        def get_currentPage(self, driver):
            try:
                currentPage = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/ul/li/strong").text
                return currentPage
            except Exception as e:
                print(e)
                driver.quit()
                
        def get_totalPage(self, driver):
            try:
                totPageCnt = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/span/span").text
                return totPageCnt
            except Exception as e:
                print(e)
                driver.quit()
        
        def get_searchList(self, driver):
            try:
                searchList = driver.find_element(By.ID, "search_list")
#                 searchList = driver.find_element(By.ID, "search_gallery")
                return searchList
            except Exception as e:
                print(e)
                driver.quit()
                
        def click_moreButton(self, driver):
            try:
                moreBtns = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[5]/div[1]/div[3]//a[@class='btn_small btn_more' and text()='더보기']")
                if len(moreBtns) > 0 :
                    print("click btn cnt = "+str(len(moreBtns)))
                    for moreBtn in moreBtns:
                        moreBtn.click()
                        time.sleep(2)
            except Exception as e:
                print(e)

        def crawling_detailPage(self, driver, searchList):
            try:
                # searchListLength = len(searchList.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[1]/table/tbody/tr/td[2]/div[2]/a")) ## 펼쳐보기에서 title Link가져오기
                searchListLength = len(searchList.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[2]/table/tbody/tr/td/div/div[4]/a")) ## 간략보기에서 title Link가져오기
                list_count = 0

                # while (list_count < result_length): change loop -> while loop to for loop
                for list_count in range(searchListLength): 
                    print("Crawling number = "+str(list_count+1)+"/"+str(searchListLength))
                #     titleLinks = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[1]/table/tbody/tr/td[2]/div[2]/a") ## 펼쳐보기
                    titleLinks = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[2]/table/tbody/tr/td/div/div[4]/a") ## 간략보기
                    titleLinks[list_count].click()
                    time.sleep(5)

                    detailInfo = dict()
                    
                    # class="box_detail_point"
                    detailPoints = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[2]/form/div[1]/*")
                    for detail in detailPoints:
                        detailInfo[detail.get_attribute("class")] = detail.text

                    # class="book_keyword"
                    keywords = []
                    if len(driver.find_elements(By.CLASS_NAME, "book_keyword")) > 0:
                        for keyword in driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[2]/form/div[2]/div[4]/div[2]/*"):
                            keywords.append(keyword.text)
                        detailInfo['book_keyword'] = ", ".join(keywords)

                    # more button click
                    self.click_moreButton(driver)
                    
                    # "content"
                    detailcontentList = []
                    contents = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[5]/div[1]/div[3]/*")
                    for content in contents:
                        detailcontentList.append(content.text)
                    detailInfo['content'] = "\n".join(detailcontentList)
                    # "URL"
                    detailInfo['url'] = driver.current_url
                    print(detailInfo)
                    driver.back()
                #     list_count+=1
                    time.sleep(2)
            
                return detailInfo
            except Exception as e:
                print(e)

    else:
        exit()

In [3]:
bookSelection = BookSelection("http://www.kyobobook.co.kr/", "python")
driver = bookSelection.set_drivers()
bookSelection.get_url(driver)
bookSelection.get_searchKeywords(driver)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/administrator/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [4]:
searchTabList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[1]/ul").text.split('\n')
subTabList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[2]/ul").text.split('\n')
categoryList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[3]/table/tbody/tr[1]/td/ul").text.split('\n')

In [5]:
print(searchTabList)
print(subTabList)
print(categoryList)

['상품(5,299)', '중고장터(532)', '스토리K(25)', '리뷰(1,262)', '책소개.목차(3,400)']
['전체 5,299', '국내도서 1,157', '외국도서 3,605', 'eBook 500', 'sam 90', '음반/영상 37', '기프트 0']
['컴퓨터/IT 1,111', '어린이(초등) 11', '경제/경영 11', '과학 7', '청소년 3', '정치/사회 3', '취업/수험서 2', '자기계발 2']


In [6]:
## 리스트 보기 개수 조절
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[2]").click()
time.sleep(1)

driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[2]/ul/li[3]").click()
time.sleep(2)

## 리스트 간략히 보기
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[3]").click()
time.sleep(1)

driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[3]/ul/li[2]/a").click()
time.sleep(2)

totalPages = bookSelection.get_totalPage(driver)
print("totPageCnt = "+str(totalPages))

totPageCnt = 89


## 페이지 처리, 상세페이지 크롤링

In [7]:
for page in range(int(totalPages)):
    currentPage = bookSelection.get_currentPage(driver)
    print("CurrentPage = "+str(currentPage))
    
    ## Crawling List    
    searchList = bookSelection.get_searchList(driver)
    ## 상세페이지 Crawling
    detailInfo = bookSelection.crawling_detailPage(driver, searchList)
    
    if currentPage != totalPages:
        driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[9]/div[1]/a[3]").click()
    else:
        print("Complete crawling of all pages!!")
        break
    time.sleep(2)

CurrentPage = 1
Crawling number = 1/60
click btn cnt = 1
{'icon': '', 'title': '혼자 공부하는 파이썬 파이썬 최신 버전 반영', 'author': '윤인성 지음 | 한빛미디어 | 2019년 06월 10일 출간', 'review': 'Klover\n총 4 중\n4 9.6 (리뷰 218개)\n클로버 리뷰쓰기', 'rank': '국내도서 주간베스트 293위 | 컴퓨터/IT 주간베스트 9위 교보문고 베스트셀러', 'book_keyword': '', 'content': '도서·상품정보 1:1 문의하기\nISBN 9791162241882(1162241888)\n쪽수 460쪽\n크기 188 * 258 * 28 mm /1001g 판형알림\n책소개\n이 책이 속한 분야\n컴퓨터/IT > 프로그래밍 언어 > Python(파이썬)\n컴퓨터/IT > 대학교재\n대학교재 > 컴퓨터\n이 책의 주제어\n#프로그래밍 #자습서 #입문자 #손코딩 #전공자\n\n혼자 해도 충분하다! 1:1 과외하듯 배우는 파이썬 프로그래밍 자습서(파이썬 최신 버전 반영)\n27명의 베타리더 검증으로, ‘함께 만든’ 입문자 맞춤형 도서\n이 책은 독학으로 프로그래밍 언어를 처음 배우려는 입문자가, 혹은 파이썬을 배우려는 입문자가 ‘꼭 필요한 내용을 제대로’ 학습할 수 있도록 구성했다. ‘무엇을’, ‘어떻게’ 학습해야 할지조차 모르는 입문자의 막연한 마음을 살펴, 과외 선생님이 알려주듯 친절하게, 그러나 핵심적인 내용만 콕콕 집어준다. 책의 첫 페이지를 펼쳐서 마지막 페이지를 덮을 때까지, 혼자서도 충분히 파이썬을 배울 수 있다는 자신감과 확신이 계속될 것이다!\n\n27명의 베타리더와 함께 구성하여 입문자에게 맞는 난이도, 분량, 학습 요소 등을 적극 반영했다. 어려운 용어와 개념은 한번 더 풀어 쓰고, 복잡한 설명은 눈에 잘 들어오는 그림으로 풀어 냈다. ‘혼자 공부해본’ 여러 입문자의 초심과 눈높이가 책 곳곳에 반영된 것이 이 책의 가장 큰 장

Crawling number = 3/60
click btn cnt = 2
{'icon': '', 'title': '1일 1로그 100일 완성 IT 지식 하드웨어,소프트웨어,통신,데이터,4가지 IT 근육으로 디지털 문해력 기르기', 'author': '브라이언 W. 커니핸 지음 | 하성창 옮김 | 인사이트 | 2021년 12월 31일 출간', 'review': '총 4 중\n4 9.4 (리뷰 3개)\n클로버 리뷰쓰기', 'rank': '국내도서 주간베스트 169위 | 컴퓨터/IT 주간베스트 5위', 'book_keyword': '', 'content': '도서·상품정보 1:1 문의하기\nISBN 9788966263301(8966263305)\n쪽수 560쪽\n크기 153 * 206 * 40 mm /649g 판형알림\n책소개\n이 책이 속한 분야\n컴퓨터/IT > IT일반 > IT교양서/IT에세이\n자기계발 > 성공/처세 > 자기관리/처세\n이 책의 주제어\n#기초부터시작하는IT수업 #IT수업 #디지털문해력 #컴퓨터과학명강의 #하드웨어 #통신 #데이터\n\n복잡한 IT 세상을 선명하게 읽는 디지털 문해력 기르기 챌린지\nIT 지식은 분명 복잡하지만 인생처럼 혼잡하지는 않다. 필요한 지식을 습득하면 막연한 불안감에서 벗어나 디지털 지구가 제시하는 도전과 기회에 적극적으로 대처할 수 있다. 네이버, 카카오, 쿠팡, 배달의 민족, 당근마켓, 토스··· 순식간에 우리 일상에 스며든 IT 기업들은 클라우드 컴퓨팅, 머신러닝, 데이터 마이닝과 같은 기술을 바탕으로 매분 매초 혁신과 성장을 거듭하고 있다. 그 가운데 인터넷, 스마트폰에 이어 2025년 글로벌 300조 시장이 예측되는 메타버스 시대가 열렸다.\n다행히도 논리적 구조와 물리적 구현으로 이뤄진 디지털 시스템의 기본 아이디어는 무게가 30톤에 달하던 초기 컴퓨터부터 MS 홀로렌즈로 접속하는 메타버스까지 동일하다. 컴퓨팅의 4가지 핵심 분야인 하드웨어, 소프트웨어, 통신, 데이터를 이해하면 어떤 복잡한 디지털 시스템이

KeyboardInterrupt: 